In [1]:
from pandas import json_normalize
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import pandas as pd
from IPython.display import display
from tqdm import tqdm, tqdm_notebook, trange
import sentencepiece as spm
import wget
import import_ipynb
import BERT

import torch
import torch.nn as nn
import torch.nn.functional as F


importing Jupyter notebook from BERT.ipynb


# Fine Tune Labeling

In [2]:
with open('train_original.json') as file:
    data = json.load(file)


In [20]:
df = json_normalize(data['documents'])

In [21]:
df.head()

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,extractive,abstractive,document_quality_scores.readable,document_quality_scores.accurate,document_quality_scores.informative,document_quality_scores.trustworthy
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...,4,3,3,3
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...,4,4,5,4
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...,4,4,4,4
3,290741794,정치,online,지역지,광양신문,small,746,2018-01-05 18:50:17,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...",12,"[2, 3, 4]","[광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화...",4,4,5,4
4,290741797,종합,online,지역지,광양신문,small,978,2018-01-05 18:52:36,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...",13,"[3, 7, 4]",[올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입...,4,4,4,4


In [4]:
df = df[['title','text','extractive']]
df.head()

,title,text,extractive
0,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...","[2, 3, 10]"
1,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...","[2, 4, 11]"
2,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...","[3, 5, 7]"
3,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...","[2, 3, 4]"
4,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...","[3, 7, 4]"


In [4]:
vocab_file = "/home/studio/바탕화면/web-crawler/kowiki/vocab_32000/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [5]:
def preprocess_data(vocab, infile, outfile):
    with open(infile) as file:
        data = json.load(file)
    df = json_normalize(data['documents'])
    df = df[['title','text','extractive']]
    label=0
    with open(outfile, 'w') as file:
        for index,ext in enumerate(tqdm(df['extractive'])):
            count=0
            for i, text in enumerate(df['text'][index]):
                try:
                    for i_ in range(len(text)):
                        sentence_=text[i_]['sentence']
                        if len(sentence_)>500:
                            sentence_ = sentence_[len(sentence_)-500:len(sentence_)]
                        text_=vocab.encode_as_pieces(sentence_)
                        if count in ext:
                            label=1
                        else:
                            label=0
                        instance = {"id":count, "text":text_, "label":label}
                        file.write(json.dumps(instance))
                        file.write("\n")
                        count+=1
                except:
                    print(index,i)
                    continue


In [6]:
preprocess_data(vocab, "train_original.json", "preprocess_train_original.json")
preprocess_data(vocab, "valid_original.json", "preprecess_valid_original.json")


100%|███████████████████████████████████| 30122/30122 [00:16<00:00, 1776.93it/s]


# 코드 정리

In [2]:
from pandas import json_normalize
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import pandas as pd
from IPython.display import display
from tqdm import tqdm, tqdm_notebook, trange
import sentencepiece as spm
import wget
import import_ipynb
import BERT

import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
vocab_file = "/home/studio/바탕화면/web-crawler/kowiki/vocab_32000/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [4]:
# binary classification
class ExtractiveClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.bert = BERT.BERT(self.config)
        # classfier
        self.projection_cls = nn.Linear(self.config.d_model, self.config.n_output, bias=False)
    
    def forward(self, inputs, segments):
        # (bs, n_enc_seq, d_hidn), (bs, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        outputs, outputs_cls, attn_probs = self.bert(inputs, segments)
        # (bs, n_output)
        logits_cls = self.projection_cls(outputs_cls)
        # (bs, n_output), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return logits_cls, attn_probs

In [5]:
class ExtDataSet(torch.utils.data.Dataset):
    def __init__(self, vocab, infile):
        self.vocab = vocab
        self.labels = []
        self.sentences = []
        self.segments = []

        line_cnt = 0
        with open(infile, "r") as f:
            for line in f:
                line_cnt += 1

        with open(infile, "r") as f:
            for i, line in enumerate(tqdm(f, total=line_cnt, desc="Loading Dataset", unit=" lines")):
                data = json.loads(line)
                self.labels.append(data["label"])
                sentence = [vocab.piece_to_id("[CLS]")] + [vocab.piece_to_id(p) for p in data["text"]] + [vocab.piece_to_id("[SEP]")]
                self.sentences.append(sentence)
                self.segments.append([i%2] * len(sentence))
    
    def __len__(self):
        assert len(self.labels) == len(self.sentences)
        assert len(self.labels) == len(self.segments)
        return len(self.labels)
    
    def __getitem__(self, item):
        return (torch.tensor(self.labels[item]),
                torch.tensor(self.sentences[item]),
                torch.tensor(self.segments[item]))

In [6]:
def EXT_collate_fn(inputs):
    labels, inputs, segments = list(zip(*inputs))

    inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
    segments = torch.nn.utils.rnn.pad_sequence(segments, batch_first=True, padding_value=0)

    batch = [
        torch.stack(labels, dim=0),
        inputs,
        segments,
    ]
    return batch

In [7]:
batch_size = 64
train_dataset = ExtDataSet(vocab, f"preprocess_train_original.json")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=EXT_collate_fn)
test_dataset = ExtDataSet(vocab, f"preprecess_valid_original.json")
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=EXT_collate_fn)

Loading Dataset: 100%|███████████| 441161/441161 [00:10<00:00, 42780.32 lines/s]


In [8]:
def eval_epoch(config, model, data_loader):
    matchs = []
    model.eval()

    n_word_total = 0
    n_correct_total = 0
    with tqdm(total=len(data_loader), desc=f"Valid") as pbar:
        for i, value in enumerate(data_loader):
            labels, inputs, segments = map(lambda v: v.to(config.device), value)

            outputs = model(inputs, segments)
            logits_cls = outputs[0]
            _, indices = logits_cls.max(1)

            match = torch.eq(indices, labels).detach()
            matchs.extend(match.cpu())
            accuracy = np.sum(matchs) / len(matchs) if 0 < len(matchs) else 0

            pbar.update(1)
            pbar.set_postfix_str(f"Acc: {accuracy:.3f}")
    return np.sum(matchs) / len(matchs) if 0 < len(matchs) else 0

In [9]:
def train_epoch(config, epoch, model, criterion_cls, optimizer, train_loader):
    losses = []
    model.train()

    with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
        for i, value in enumerate(train_loader):
            labels, inputs, segments = map(lambda v: v.to(config.device), value)

            optimizer.zero_grad()
            outputs = model(inputs, segments)
            logits_cls = outputs[0]

            loss_cls = criterion_cls(logits_cls, labels)
            loss = loss_cls

            loss_val = loss_cls.item()
            losses.append(loss_val)

            loss.backward()
            optimizer.step()

            pbar.update(1)
            pbar.set_postfix_str(f"Loss: {loss_val:.3f} ({np.mean(losses):.3f})")
    return np.mean(losses)

In [10]:
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

In [11]:
config = Config({
    "n_enc_vocab": len(vocab), # vocab 크기
    "n_enc_seq": 512,          # 글자 최대 길이 
    "n_seg_type": 2,           # Segment Embedding Type  
    "n_layer":6,             # layer 캣수
    "d_model": 512,            # hidden layer 
    "i_pad": 0,                # padding 값
    "d_ff": 1024,              # feedforward layer에 들어갈 차원의 크기
    "n_head": 6,              # attention 개수
    "d_head": 64,              # attention 차원 
    "dropout": 0.1,            # dropout
    "layer_norm_epsilon": 1e-12 # 정규화
})
print(config)

{'n_enc_vocab': 32007, 'n_enc_seq': 512, 'n_seg_type': 2, 'n_layer': 6, 'd_model': 512, 'i_pad': 0, 'd_ff': 1024, 'n_head': 6, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12}


In [12]:
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.n_output = 2
print(config)

learning_rate = 5e-5
n_epoch = 10

{'n_enc_vocab': 32007, 'n_enc_seq': 512, 'n_seg_type': 2, 'n_layer': 6, 'd_model': 512, 'i_pad': 0, 'd_ff': 1024, 'n_head': 6, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12, 'device': device(type='cuda'), 'n_output': 2}


In [13]:
def train(model):
    model.to(config.device)

    criterion_cls = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    best_epoch, best_loss, best_score = 0, 0, 0
    losses, scores = [], []
    for epoch in range(n_epoch):
        loss = train_epoch(config, epoch, model, criterion_cls, optimizer, train_loader)
        score = eval_epoch(config, model, test_loader)

        losses.append(loss)
        scores.append(score)

        if best_score < score:
            best_epoch, best_loss, best_score = epoch, loss, score
            print("save : ")
            torch.save(model.state_dict(), f'finetune_model{best_score}.pt')
    print(f">>>> epoch={best_epoch}, loss={best_loss:.5f}, socre={best_score:.5f}")
    return losses, scores

In [14]:
torch.manual_seed(0)
model = ExtractiveClassification(config).to(config.device)
model.load_state_dict(torch.load('finetune_model.pt'))
losses, scores = train(model)
torch.save(model.state_dict(), f'finetune_model_last.pt')

Valid: 100%|████████████████████| 6894/6894 [35:38<00:00,  3.22it/s, Acc: 0.840]


save : 


Valid: 100%|████████████████████| 6894/6894 [35:45<00:00,  3.21it/s, Acc: 0.840]


save : 


Train(7):  67%|████  | 36452/54620 [28:31<14:12, 21.30it/s, Loss: 0.296 (0.353)]


KeyboardInterrupt: 

In [15]:
torch.save(model.state_dict(), 'finetune_model_last.pt')

In [15]:
model

MovieClassification(
  (bert): BERT(
    (encoder): Encoder(
      (enc_emb): Embedding(32007, 512)
      (pos_emb): Embedding(513, 512)
      (seg_emb): Embedding(2, 512)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): MultiHeadAttention(
            (W_Q): Linear(in_features=512, out_features=384, bias=True)
            (W_K): Linear(in_features=512, out_features=384, bias=True)
            (W_V): Linear(in_features=512, out_features=384, bias=True)
            (scaled_dot_attn): ScaledDotProductAttention(
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (linear): Linear(in_features=384, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
          (pos_ffn): PoswiseFeedForwardNet(
            (conv1): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
            (conv2): Conv1d(1024, 512, kernel_size=(1,

# 테스트

In [3]:
from pandas import json_normalize
import os
import re
import numpy as np
import math
import matplotlib.pyplot as plt
import json
import pandas as pd
from IPython.display import display
from tqdm import tqdm, tqdm_notebook, trange
import sentencepiece as spm
import wget
import import_ipynb
import BERT

import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
vocab_file = "/home/studio/바탕화면/web-crawler/kowiki/vocab_32000/kowiki.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)

True

In [3]:
class ExtractiveClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.bert = BERT.BERT(self.config)
        # classfier
        self.projection_cls = nn.Linear(self.config.d_model, self.config.n_output, bias=False)
    
    def forward(self, inputs, segments):
        # (bs, n_enc_seq, d_hidn), (bs, d_hidn), [(bs, n_head, n_enc_seq, n_enc_seq)]
        outputs, outputs_cls, attn_probs = self.bert(inputs, segments)
        # (bs, n_output)
        logits_cls = self.projection_cls(outputs_cls)
        # (bs, n_output), [(bs, n_head, n_enc_seq, n_enc_seq)]
        return logits_cls, attn_probs

In [4]:
class Config(dict): 
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__

    @classmethod
    def load(cls, file):
        with open(file, 'r') as f:
            config = json.loads(f.read())
            return Config(config)

In [5]:
config = Config({
    "n_enc_vocab": len(vocab), # vocab 크기
    "n_enc_seq": 512,          # 글자 최대 길이 
    "n_seg_type": 2,           # Segment Embedding Type  
    "n_layer":6,             # layer 캣수
    "d_model": 512,            # hidden layer 
    "i_pad": 0,                # padding 값
    "d_ff": 1024,              # feedforward layer에 들어갈 차원의 크기
    "n_head": 6,              # attention 개수
    "d_head": 64,              # attention 차원 
    "dropout": 0.1,            # dropout
    "layer_norm_epsilon": 1e-12 # 정규화
})
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
config.n_output = 2
print(config)

{'n_enc_vocab': 32007, 'n_enc_seq': 512, 'n_seg_type': 2, 'n_layer': 6, 'd_model': 512, 'i_pad': 0, 'd_ff': 1024, 'n_head': 6, 'd_head': 64, 'dropout': 0.1, 'layer_norm_epsilon': 1e-12, 'device': device(type='cuda'), 'n_output': 2}


In [26]:
torch.manual_seed(0)
test_model = ExtractiveClassification(config).to(config.device)
test_model.load_state_dict(torch.load('finetune_model.pt'))

<All keys matched successfully>

In [7]:
test_model

ExtractiveClassification(
  (bert): BERT(
    (encoder): Encoder(
      (enc_emb): Embedding(32007, 512)
      (pos_emb): Embedding(513, 512)
      (seg_emb): Embedding(2, 512)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): MultiHeadAttention(
            (W_Q): Linear(in_features=512, out_features=384, bias=True)
            (W_K): Linear(in_features=512, out_features=384, bias=True)
            (W_V): Linear(in_features=512, out_features=384, bias=True)
            (scaled_dot_attn): ScaledDotProductAttention(
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (linear): Linear(in_features=384, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
          (pos_ffn): PoswiseFeedForwardNet(
            (conv1): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
            (conv2): Conv1d(1024, 512, kernel_siz

In [8]:
def find_max(text):
    text_index = []
    text_value = []
    for i in range(10):
        t1,t2 = output_index(test_news(text_split(text))),text_split(text)
        text_index.append(t1[0])
        text_value.append(sum(t1[0]))
    return text_index[text_value.index(min(text_value))]

def test_news(text):
    answer=[]
    for i,text_ in enumerate(text):
        sentence = [vocab.piece_to_id("[CLS]")] + [vocab.piece_to_id(p) for p in text_] + [vocab.piece_to_id("[SEP]")]
        segments=[i%2] * len(sentence)
        sentence = torch.tensor(sentence).expand(1,-1).to(config.device)
        segments = torch.tensor(segments).expand(1,-1).to(config.device)
        answer.append(test_model(sentence, segments)[0][0][1])
    return answer
def text_split(text):
    trim_text=[]
    for s in text.split('.'):
        cut_text = s.replace('\n','')+'.'
        if cut_text != '.':
            trim_text.append(cut_text)
    return trim_text

def output_index(value_list):
    answer=[]
    ext_value = 0
    if len(value_list)>15:
        count = len(value_list)//5
    else:
        count = 3
    for i in range(3):
        max_value = value_list.index(max(value_list))
        ext_value += max(value_list)
        answer.append(max_value)
        value_list.pop(max_value)
        value_list.insert(max_value,float('-inf'))
    return [answer,ext_value]

def concat_text(index_list, sentence):
    index_list.sort()
    text = ''
    for t in index_list:
        text += sentence[t]
    return text

def ROUGUE(index_list, sentence):
    index_list.sort()
    text = []
    for t in index_list:
        text.append(sentence[t])
    return text

def EXTSentence(sentence):
    return concat_text(find_max(sentence),text_split(sentence))
#     return concat_text(output_index(test_news(text_split(a)))[0],text_split(a))


In [9]:
a="""

국방부가 서울 용산에 있는 합동참모본부를 남태령으로 이전하는 비용 중 55억9000만원을 내년도 예산안에 포함하려 한 것으로 나타났다. 국방부·합참은 합참 이전 비용을 내년도 예산안에 편성해달라는 요구를 하지 않았다고 주장한 것으로 알려졌다. 실제 내년도 예산안에 반영된 합참 이전 비용은 전혀 없다. 예비타당성 조사까지 면제하며 내년도 예산안에 210억원이 편성된 방위사업청 이전과 달리 정부가 합참 이전 의지가 없다는 지적이 제기된다.

4일 국회 국방위원회 소속 윤후덕 더불어민주당 의원이 합참으로부터 받은 자료를 보면, 합참은 내년도 예산안에 합참 이전 비용 중 55억9000만원을 반영해달라고 요구할 계획을 세웠다. 합참은 2024년 이후 합참 이전 예산에 건축비 2100억여원 등 추가 비용이 들 것으로 보고, 사전에 필요한 예산 일부를 먼저 반영해달라고 요청할 계획이었다. 그러나 기획재정부 심사 등을 거치는 과정에서 해당 요구안은 실제로 반영되지 않았다.

윤 의원에 따르면 국방부와 합참은 “합참 이전 비용을 내년도 예산안에 요구하지 않았다”는 입장을 의원실에 전했다. 지난달 19일 국회 국방위원회 전체회의에서는 합참 이전 관련 비용이 내년도 예산안에 왜 반영되지 않았냐는 정성호 민주당 의원 질의에 김승겸 합참의장이 “사업 타당성 조사가 이뤄지지 않았다”고 답했다. 국방부 관계자는 기자에게도 “합참 이전 관련 내년도 예산안으로 요청한 바 없다. 사업타당성 조사가 끝난 후 요구할 예정”이라고 설명했다.

이는 예비타당성 조사를 면제하며 내년도 예산안에 210억원을 편성한 방사청 이전 사업과 대조적이다. 윤석열 정부 출범 후 예비타당성 조사가 면제된 사업은 총 16건인데, 방사청 이전 외에도 육군 관사 등 시설 개선 사업이 6건이 추가로 포함됐지만 대통령실 용산 이전 후 필수적인 합참 이전 예산은 반영되지 않았다.

윤후덕 의원은 “육군 관사, 방사청 이전보다 합참 이전이 덜 중요한 사업인지 묻지 않을 수 없다”며 “합참 이전이 내년 예산이 아니라 2024년 예산부터 반영되면 현 정부 내 완전 이전이 어려워 안보 공백과 혼란이 계속될까 우려된다”고 말했다.
"""

In [10]:
EXTSentence(a)

'4일 국회 국방위원회 소속 윤후덕 더불어민주당 의원이 합참으로부터 받은 자료를 보면, 합참은 내년도 예산안에 합참 이전 비용 중 55억9000만원을 반영해달라고 요구할 계획을 세웠다.윤 의원에 따르면 국방부와 합참은 “합참 이전 비용을 내년도 예산안에 요구하지 않았다”는 입장을 의원실에 전했다.윤후덕 의원은 “육군 관사, 방사청 이전보다 합참 이전이 덜 중요한 사업인지 묻지 않을 수 없다”며 “합참 이전이 내년 예산이 아니라 2024년 예산부터 반영되면 현 정부 내 완전 이전이 어려워 안보 공백과 혼란이 계속될까 우려된다”고 말했다.'

In [28]:
EXTSentence(a)

'윤석열 대통령이 12일(한국시각) 밤 10에 화상으로 열리는 ‘제2차 글로벌 코로나19 정상회의’에 참석한다.화상형식이기는 하나 다자외교 무대에 데뷔하는 것이다.의제는 백신 접종, 진단검사와 치료제 접근 확대, 보건안보 강화와 미래 재난 방지를 위한 방안 등이다.'

In [123]:
EXTSentence(a)

'서울=연합뉴스) 조민정 김영신 기자 = 방역당국이 확진자 수 감소 추세와 일반의료체계로의 전환을 고려해 지난달부터 코로나19 병상 2만656개를 지정 해제했다고 25일 밝혔다.다만 고위험 확진자의 빠른 입원·치료를 위해 중등증 환자를 위한 거점보유병상도 일부 유지하기로 했다.일각에서는 병상이 빠르게 감축되면서 하반기 재유행시 신속한 대응에 어려움을 겪을 수 있다는 지적이 나온다.'

In [121]:
EXTSentence(a)

'친러 국가인 중국에서 러시아를 향한 비판이 나온 것은 이례적이다. 11일(현지시간) 영국 가디언에 따르면 가오위셩 전 우크라이나 주재 중국대사는 중국 사회과학원 온라인 세미나에서 “러시아의 (우크라이나 침공) 전쟁이 실패하고 있다”고 진단했다. 이번 전쟁으로 우크라이나가 오히려 러시아의 영향권에서 벗어나 서방에 더 가까워졌다고 그는 주장했다.'

In [24]:
EXTSentence(a)

'윤석열 대통령이 12일(한국시각) 밤 10에 화상으로 열리는 ‘제2차 글로벌 코로나19 정상회의’에 참석한다.윤석열 대통령이 12일 오후 서울 용산 대통령실 청사에서 추경 예산안 편성을 위한 첫 국무회의를 주재하고 있다.지난 10일 취임한 윤 대통령의 다자 정상회의 참가는 이번이 처음이다.'

In [82]:
text_split(EXTSentence(a))

[' 친러 국가인 중국에서 러시아를 향한 비판이 나온 것은 이례적이다.',
 '11일(현지시간) 영국 가디언에 따르면 가오위셩 전 우크라이나 주재 중국대사는 중국 사회과학원 온라인 세미나에서 “러시아의 (우크라이나 침공) 전쟁이 실패하고 있다”고 진단했다.',
 '가오 전 대사는 “(우크라이나에서) 러시아의 정치ㆍ경제ㆍ군사ㆍ외교력이 급격히 약해지고 고립되는 새로운 세계 질서가 나타날 가능성이 높다”고 내다봤다.']

In [11]:
with open('train_original.json') as file:
    data = json.load(file)


In [12]:
df = json_normalize(data['documents'])

In [13]:
df = df[['title','text','extractive']]
df.head()

,title,text,extractive
0,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...","[2, 3, 10]"
1,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...","[2, 4, 11]"
2,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...","[3, 5, 7]"
3,"농업인 역량 강화, 새해 실용교육 실시","[[{'index': 0, 'sentence': '11~24일, 매실·감·참다래 등...","[2, 3, 4]"
4,타이완 크루즈관광객 4천여명‘전남’온다,"[[{'index': 0, 'sentence': '홍콩 크루즈선사‘아쿠아리우스’ 4...","[3, 7, 4]"


In [27]:
count=0
answer=[]
for index,ext in enumerate(tqdm(df['extractive'])):
    if count==200: break
    line=0
    for i, text in enumerate(df['text'][index]):
        for i_ in range(len(text)):
            if line in ext:
                sentence_=text[i_]['sentence']
#                 sentence_=vocab.encode_as_pieces(sentence_)
                answer.append(sentence_)
            line+=1
    count+=1

  0%|                                  | 200/243983 [00:00<00:01, 123561.76it/s]


In [28]:
count=0
valid=[]
for index,ext in enumerate(tqdm(df['extractive'])):
    if count==200: break
    sentence=''
    for i, text in enumerate(df['text'][index]):
        for i_ in range(len(text)):
            sentence_=text[i_]['sentence']
            sentence_=re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", sentence_)
            if len(sentence_)>500:
                sentence_ = sentence_[len(sentence_)-500:len(sentence_)]
            sentence += sentence_+'.\n'
    valid += text_split(EXTSentence(sentence))
    count+=1

  0%|                                   | 200/243983 [02:10<44:20:55,  1.53it/s]


In [14]:
sentence

'ha당 조사료 400만원작물별 차등 지원\n이성훈 sinawihanmailnet\n전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를 적극 추진키로 했다\n쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다\n올해 전남의 논 다른 작물 재배 계획면적은 전국 5만ha의 약 21인 1만 698ha로 세부시행지침을 확정 시군에 통보했다\n지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a300평 이상 벼 이외 다른 작물을 재배한 농업인이다\n지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추 무 배추 인삼 대파 등 수급 불안 품목은 제외된다\n농지의 경우도 이미 다른 작물 재배 의무가 부여된 간척지 정부매입비축농지 농진청 시범사업 경관보전 직불금 수령 농지 등은 제외될 예정이다\nha3000평당 지원 단가는 평균 340만원으로 사료작물 400만원 일반작물은 340만원 콩팥 등 두류작물은 280만원 등이다\n벼와 소득차와 영농 편이성을 감안해 작물별로 차등 지원된다\n논에 다른 작물 재배를 바라는 농가는 오는 22일부터 2월 28일까지 농지 소재지 읍면동사무소에 신청해야 한다\n전남도는 도와 시군에 관련 기관과 농가 등이 참여하는논 타작물 지원사업 추진협의회를 구성 지역 특성에 맞는 작목 선정 및 사업 심의 등을 본격 추진할 방침이다\n최향철 전라남도 친환경농업과장은 최근 쌀값이 다소 상승추세에 있으나 매년 공급과잉에 따른 가격 하락으로 쌀농가에 어려움이 있었다며쌀 공급과잉을 구조적으로 해결하도록 논 타작물 재배 지원사업에 많이 참여해주길 바란다고 말했다\n'

In [13]:
len(answer)

150

In [18]:
len(valid)

150

In [25]:
sentence

'이성훈 sinawi@hanmail.net\n여수광양항만공사(사장 방희석, 이하 공사)는 2018년도 승진 및 전보 인사를 단행했다고 지난 26일 밝혔다.\n공사는 광양항 활성화 및 물동량 창출을 위해‘항만운영팀’, ‘글로벌마케팅팀’, ‘물류단지팀’의 인력을 보강하고자 인사를 실시하게 됐다고 설명했다.\n특히 정부 정책의 적극적인 이행과 일자리 창출 등 사회적 가치 실현을 위해‘가치경영팀’을 신설한 것도 이번 인사의 요인이라고 덧붙였다.\n*2018년도 승진·전보 인사\n<승진> ◇ 1급(실·팀장) △항만운영팀장 김한춘 ◇ 2급(부장) △항만개발팀장 고영찬 △재무회계팀 박정철 △물류단지팀 장방식 ◇ 3급(차장) △가치경영팀 박상우 △인재육성팀 박진수 △기획조정실 양헌모 △항만시설팀 고훈정 ◇ 4급(과장) △재무회계팀 임지현 △항만개발팀 권규하 ◇ 6급(주임) △경영지원팀 최민경 △항만운영팀 조현성 △가치경영팀 김세라 △글로벌마케팅팀 유창기 △항만운영팀 배순길 △물류단지팀 조보라 △여수사업소 채성석 ◇ 7급(가) △가치경영팀 양대송\n<전보> ◇ 1급(실·팀장) △재무회계팀장 최연철 △항만개발팀 기술자문위원 권영원 ◇ 2급(부장) △기획조정실장 선정덕 △가치경영팀장 정기철 △글로벌마케팅팀장 백정원 △미래사업팀 조성래 △항만운영팀 천철웅 ◇ 3급(차장) △인재육성팀 윤승재 △기획조정실 이병홍 △기획조정실 김일영 △가치경영팀 박 신 △항만운영팀 손정국 △물류단지팀 권용재 △여수사업소 권석록 △감사팀 고우권 ◇ 4급(과장) △감사팀 정연형 △기획조정실 주성구 △항만운영팀 봉만식 △물류단지팀 조령래 ◇ 5급(대리) △미래사업팀 박경민 △가치경영팀 정혜성 △가치경영팀 허 철 △항만운영팀 나병제 ◇ 6급(주임) △감사팀 최진혁 △기획조정실 김민주 △경영지원팀 김지성 △가치경영팀 박여진 △여수사업소 배수현\n'

In [19]:
from rouge import Rouge
rougue_test = Rouge()
rougue_test.get_scores(valid[:len(answer)],answer,avg=True)

{'rouge-1': {'r': 0.36190033273608163,
  'p': 0.36686092248347146,
  'f': 0.3626422845625625},
 'rouge-2': {'r': 0.3346825396825397,
  'p': 0.3321825396825397,
  'f': 0.33329364896237507},
 'rouge-l': {'r': 0.36190033273608163,
  'p': 0.36686092248347146,
  'f': 0.3626422845625625}}

In [15]:
from rouge import Rouge
rougue_test = Rouge()
rougue_test.get_scores(valid[:len(answer)],answer,avg=True)

{'rouge-1': {'r': 0.39083613625586333,
  'p': 0.3947329808795855,
  'f': 0.39102006440191206},
 'rouge-2': {'r': 0.34512690525981526,
  'p': 0.34610480978541963,
  'f': 0.34544864838364303},
 'rouge-l': {'r': 0.38876567745684504,
  'p': 0.39226294833215364,
  'f': 0.3888101790595631}}

In [29]:
from rouge import Rouge
rougue_test = Rouge()
rougue_test.get_scores(valid[:len(answer)],answer,avg=True)

{'rouge-1': {'r': 0.36939979530311606,
  'p': 0.3769749487968717,
  'f': 0.3709373223607037},
 'rouge-2': {'r': 0.316590138857307,
  'p': 0.3182712260484179,
  'f': 0.31704829620751407},
 'rouge-l': {'r': 0.36767265883963485,
  'p': 0.3750126258196026,
  'f': 0.36919628504041263}}